In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, get_compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async
set_global_seed(42)

2023-10-22 10:30:01.036269: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 10:30:01.562708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
data_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")

data = xr.load_dataset(data_path)

In [4]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [5]:
set_global_seed(42)
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [6]:
test_data

<xarray.Dataset>
Dimensions:      (channels: 30, samples: 147, epochs: 200, participant: 5,
                  labels: 5)
Coordinates:
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 140 141 142 143 144 145 146
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * participant  (participant) object '0025' '0005' '0024' '0003' '0014'
  * labels       (labels) object 'confirmation' 'decision' ... 'response'
    stim         (participant, epochs) float64 2.0 1.0 1.0 1.0 ... nan 2.0 nan
    resp         (participant, epochs) object 'resp_left' 'resp_right' ... ''
    rt           (participant, epochs) float64 0.942 1.003 1.129 ... 0.865 nan
    cue          (participant, epochs) object 'SP' 'AC' 'AC' 'AC' ... '' 'AC' ''
    movement     (participant, epochs) object 'stim_right' 'stim_left' ... ''
    trigger      (participant, epochs) object 'SP/stim_right/resp_left' ... ''
Data variables:
    data         (participant, epochs, labels, channels, samples) float64 nan...

In [ ]:
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [ ]:
model = SAT1TopologicalConv(5, 8, len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

In [4]:
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**get_compile_kwargs())

2023-10-22 10:26:40.012988: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 10:26:40.037737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 10:26:40.037839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 10:26:40.040625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-22 10:26:40.040705: I tensorflow/compile

In [5]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20


2023-10-22 10:26:47.204611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-22 10:26:48.092737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-22 10:26:48.397695: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556f9a0f24b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-22 10:26:48.397729: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-22 10:26:48.403760: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-22 10:26:48.508515: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

757/757 [==============================] - 16s 16ms/step - loss: 3.7734 - accuracy: 0.7229 - val_loss: 0.5545 - val_accuracy: 0.7999
Epoch 2/20
757/757 [==============================] - 12s 16ms/step - loss: 2.9004 - accuracy: 0.7886 - val_loss: 0.5272 - val_accuracy: 0.8085
Epoch 3/20
757/757 [==============================] - 12s 16ms/step - loss: 2.5540 - accuracy: 0.8105 - val_loss: 0.5258 - val_accuracy: 0.8080
Epoch 4/20
757/757 [==============================] - 12s 16ms/step - loss: 2.2762 - accuracy: 0.8333 - val_loss: 0.5515 - val_accuracy: 0.8025
Epoch 5/20
757/757 [==============================] - 12s 15ms/step - loss: 2.0113 - accuracy: 0.8515 - val_loss: 0.5466 - val_accuracy: 0.8135
Epoch 6/20
757/757 [==============================] - 12s 15ms/step - loss: 1.8760 - accuracy: 0.8610 - val_loss: 0.6178 - val_accuracy: 0.8067
Epoch 7/20
757/757 [==============================] - 12s 15ms/step - loss: 1.6759 - accuracy: 0.8744 - val_loss: 0.6377 - val_accuracy: 0.8120
Epo

(<keras.src.callbacks.History at 0x7f15c6742790>,
 {'confirmation': {'precision': 0.8840579710144928,
   'recall': 0.8356164383561644,
   'f1-score': 0.8591549295774648,
   'support': 438},
  'decision': {'precision': 0.8845700824499411,
   'recall': 0.8543799772468714,
   'f1-score': 0.869212962962963,
   'support': 879},
  'encoding': {'precision': 0.8340659340659341,
   'recall': 0.863481228668942,
   'f1-score': 0.8485187255449973,
   'support': 879},
  'pre-attentive': {'precision': 0.8046272493573264,
   'recall': 0.7408284023668639,
   'f1-score': 0.7714109673444239,
   'support': 845},
  'response': {'precision': 0.8173374613003096,
   'recall': 0.9010238907849829,
   'f1-score': 0.8571428571428571,
   'support': 879},
  'accuracy': 0.8403061224489796,
  'macro avg': {'precision': 0.8449317396376008,
   'recall': 0.839065987484765,
   'f1-score': 0.8410880885145412,
   'support': 3920},
  'weighted avg': {'precision': 0.8408796183703355,
   'recall': 0.8403061224489796,
   'f1-

In [5]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    gen_kwargs={"shape_topological": True},
)

Epoch 1/20


2023-10-20 08:52:35.229254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-20 08:52:36.664603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-20 08:52:37.083859: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5592c2025fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-20 08:52:37.083910: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-20 08:52:37.113636: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-20 08:52:37.304406: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

757/757 [==============================] - 17s 15ms/step - loss: 3.4880 - accuracy: 0.7525 - val_loss: 0.5567 - val_accuracy: 0.8051
Epoch 2/20
757/757 [==============================] - 12s 15ms/step - loss: 2.5864 - accuracy: 0.8161 - val_loss: 0.4801 - val_accuracy: 0.8333
Epoch 3/20
757/757 [==============================] - 12s 15ms/step - loss: 2.2729 - accuracy: 0.8371 - val_loss: 0.4875 - val_accuracy: 0.8382
Epoch 4/20
757/757 [==============================] - 12s 15ms/step - loss: 2.0235 - accuracy: 0.8526 - val_loss: 0.4539 - val_accuracy: 0.8514
Epoch 5/20
757/757 [==============================] - 12s 15ms/step - loss: 1.8367 - accuracy: 0.8661 - val_loss: 0.4951 - val_accuracy: 0.8496
Epoch 6/20
757/757 [==============================] - 12s 15ms/step - loss: 1.6872 - accuracy: 0.8819 - val_loss: 0.4913 - val_accuracy: 0.8407
Epoch 7/20
245/245 [==============================] - 3s 11ms/step


(<keras.src.callbacks.History at 0x7f8f06139110>,
 {'confirmation': {'precision': 0.854586129753915,
   'recall': 0.8701594533029613,
   'f1-score': 0.8623024830699774,
   'support': 439},
  'decision': {'precision': 0.8778877887788779,
   'recall': 0.9078498293515358,
   'f1-score': 0.8926174496644295,
   'support': 879},
  'encoding': {'precision': 0.8289334741288279,
   'recall': 0.8981693363844394,
   'f1-score': 0.8621636463481605,
   'support': 874},
  'pre-attentive': {'precision': 0.8678977272727273,
   'recall': 0.7196702002355713,
   'f1-score': 0.7868641339343206,
   'support': 849},
  'response': {'precision': 0.8630887185104053,
   'recall': 0.8964732650739476,
   'f1-score': 0.8794642857142858,
   'support': 879},
  'accuracy': 0.8581632653061224,
  'macro avg': {'precision': 0.8584787676889507,
   'recall': 0.8584644168696911,
   'f1-score': 0.8566823997462347,
   'support': 3920},
  'weighted avg': {'precision': 0.8588812978858138,
   'recall': 0.8581632653061224,
   'f

In [ ]:
results = k_fold_cross_validate(
    data,
    model,
    5,
    normalization_fn=norm_dummy,
    train_kwargs={
        "logs_path": Path("logs/cnn_performance"),
        "additional_name": "3D+unprocessed",
    },
    gen_kwargs={"shape_topological": True},
)
print_results(results)

In [7]:
model.save("models/cnn_base_test")

INFO:tensorflow:Assets written to: models/cnn_base_test/assets


INFO:tensorflow:Assets written to: models/cnn_base_test/assets


In [7]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 770, 30, 1)]      0         
                                                                 
 masking (Masking)           (None, 770, 30, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 746, 30, 64)       1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 373, 30, 64)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 357, 30, 128)      139392    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 178, 30, 128)      0         
 g2D)                                                        